In [ ]:
# 1. 마운트 설정
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2. 현재 작업디렉터리 연결
import os
os.chdir("drive/My Drive/py_app/")

In [ ]:
# 3. 작업디렉터리가 가진 목록을 화면에 표시
!ls

# 주요 라이브러리 로드 및 시각화 선수작업

In [ ]:
# 선수작업2-1 : 주요 라이브러리 로드
import numpy as np
import scipy as sp
import pandas as pd

In [ ]:
# 선수작업2-2

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

mpl.rcParams['axes.unicode_minus'] = False  # -기호 깨짐방지

In [ ]:
# 선수작업2-3

# 나눔폰트 유무 확인
sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")
print(sys_font)

nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}")

In [ ]:
# 선수작업2-4

nanum_font

In [ ]:
# 선수작업2-5
# 나눔폰트 없는 경우 설치. 있는 경우 skip
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [ ]:
# 선수작업2-5-1
# fm._rebuild() 에러발생시 실행후 런타임 다시 시작
# Cleaning the matplotlib cache directory
!rm ~/.cache/matplotlib -rf

In [ ]:
# 선수작업2-6
# 한글폰트 지정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
# fm._rebuild()

# 그래프에 retina display 적용

%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정

plt.rc('font', family='NanumBarunGothic')

기상청 api.kma.go.kr에서 실시간 기상데이터 다운로드 후 데이터프레임으로 변환

In [ ]:
# 0. 호출url생성
from pytz import timezone
from datetime import datetime
import urllib.request as req

# inf=관측종류, stn=지점번호, tm=yyyymmddhh00
url = ""
stn = ""
inf = ["SFC"]
tm = ""
hh = ["00", "03", "06", "09", "12", "15", "18", "21"]
help = 1

now_datetime = datetime.now(timezone("Asia/Seoul"))
# now_datetime
tm = now_datetime.strftime("%Y%m%d") + input("시간입력(00~21):") + "00"
url = "http://api.kma.go.kr/url/stn_inf.php?inf=" + input("관측지점입력(SFC):")
url += "&stn=&tm=" + tm + "&help=1"
url

In [ ]:
data_fn = "data/" + tm + "_" + inf[0] + ".csv"
data_fn

In [ ]:
for f1 in inf:
  data_fn = "data/" + tm + "_" + f1 + ".csv"
  # req.urlretrieve(url, data_fn)
  # 아래 URL은 API 호출URL 예시입니다.(inf=관측종류, stn=지점번호, tm=yyyymmddhh00)
  print(data_fn)
print("done")

In [ ]:
# 1. urllib.request.urlopen을 사용한 다운로드
# 기상청 api.kma.go.kr에서 실시간 기상데이터 다운로드
# 1. openAPI 데이터 다운로드
import urllib.request as req

data = "data/20230516_test.csv"
# 아래 URL은 API 호출URL 예시입니다.(inf=관측종류, stn=지점번호, tm=yyyymmddhh00)
url = "http://api.kma.go.kr/url/stn_inf.php?inf=SFC&stn=&tm=202305090900&help=1"
req.urlretrieve(url, data)
print("done")

In [ ]:
# 2. 다운로드 받은 파일 읽어서 변수명과 데이터 문자열 추출
i = 0
head_str = ""
data_str = ""
with open('data/20230516_test.csv', 'r', encoding="cp949") as f:
  for line in f:
    i += 1
    if i == 19:
      head_str = line
    elif i >= 21:
      data_str += line
print(head_str)  # 변수명 문자열
print(data_str)  # 데이터 문자열

In [ ]:
head_str


In [ ]:
# 3. 변수명 문자열을 리스트로 변환

hstr = head_str.split()
hstr[1] = "STN_ID"
hstr = hstr[1:]
hstr


In [ ]:
data_str

In [ ]:
# 4. 데이터 문자열을 파일로 저장
with open("data/test_data.txt", "w") as f:
    f.write(data_str)

In [ ]:
# 5. 데이터 문자열 파일을 읽어서 마지막데이터 제거
datas = []
row_data = []
with open('data/test_data.txt', 'r') as f:
  for line in f:
    row_data = line.split()
    datas.append(row_data)
datas.pop()  # 마지막 원소 제거
datas

In [ ]:
# 6. 변수명의 개수확인
len(hstr)

In [ ]:
# 각 데이터마다 값의 개수 확인
[(i,len(x)) for i, x in enumerate(datas)]

In [ ]:
# 7. 정선군 데이터 값 오류 수정
len(datas[53])
datas[53][11] = "Jeongseongun"
del datas[53][12]
datas[53]

In [ ]:
# 7. 강진군 데이터 값 오류 수정
len(datas[73])
datas[73][11] = "Gangjingun"
del datas[73][12]
datas[73]

In [ ]:
[(i,len(x)) for i, x in enumerate(datas)]

In [ ]:
# 8. df_sfc 데이터 프레임생성
import pandas as pd

df_sfc = pd.DataFrame(data=datas, columns=hstr)
df_sfc

In [ ]:
# df_sfc데이터프레임 파일로 저장
df_sfc.to_csv("data/test_data_df.csv", index=False)

In [ ]:
# test_data_df.csv 데이터파일 로드
df_sfc = pd.read_csv("data/test_data_df.csv")

In [ ]:
# 9. df_sfc데이터프레임에서 각변수의 타입확인
df_sfc.dtypes

In [ ]:
df_sfc.columns


In [ ]:
# 10. 문자열을 숫자타입으로 변환
df_sfc[['LON', 'LAT', 'HT', 'HT_PA', 'HT_TA', 'HT_WD', 'HT_RN']] = df_sfc[['LON', 'LAT', 'HT', 'HT_PA', 'HT_TA', 'HT_WD', 'HT_RN']].apply(pd.to_numeric)

In [ ]:
df_sfc.dtypes

In [ ]:
df_sfc

In [ ]:
df_sfc[df_sfc["HT"] >= 200]

In [ ]:
df_202306_hw = pd.read_excel("data/ISSUE_HW_DAY_2023-06_2023-06_2023.xls.xlsx")

In [ ]:
df_202306_hw

In [ ]:
df_202306_hw.shape

In [ ]:
df_202306_hw.columns


In [ ]:
df_202306_hw.dtypes

In [ ]:
# 지점이 "서울(108)"인 데이터 추출
df_202306_hw[df_202306_hw['지점']=="서울(108)"]

In [ ]:
# 지점에 "서울"이 포함된 데이터 추출
# 포함문자열: str.contains("문자열"),  str.contains("문자열1|문자열2")
df_202306_hw[df_202306_hw['지점'].str.contains("서울")]

In [ ]:
df_202306_hw[["일시", "지점", "폭염여부(O/X)"]].groupby("폭염여부(O/X)").count()

In [ ]:
df_202306_hw[df_202306_hw["폭염여부(O/X)"] == "O"][["일시", "지점", "최고기온(°C)"]]


In [ ]:
df_202306_hw[df_202306_hw["최고기온(°C)"].notnull()]

In [ ]:
df_hw_0 = df_202306_hw[(df_202306_hw["폭염여부(O/X)"] == "O") & (df_202306_hw["최고기온(°C)"].notnull())]
df_hw_0

In [ ]:
df_hw_0[["일시", "지점", "최고기온(°C)"]]

In [ ]:
list_lc = df_hw_0["지점"].values.tolist()
list_lc

In [ ]:
# 시계열 그래프 플롯
df_202306_hw[df_202306_hw['지점']=="양주(598)"].plot(x="일시", y="최고기온(°C)")

In [ ]:
df_202306_hw[df_202306_hw['지점'] == "전주(146)"].plot(x="일시", y="최고기온(°C)")

In [ ]:
df_202306_hw[df_202306_hw['일시'] == "2023-06-21"]

In [ ]:
list_lc

In [ ]:
# 8개의 지점을 각각의 시계열 그래프로 플롯
for x in list_lc:
  df_202306_hw[df_202306_hw['지점'] == x].plot(x="일시", y="최고기온(°C)")

In [ ]:
# 1개의 그래프에 여러개의 시계열 그래프 플롯 - 서브플롯 사용
fig = plt.figure(figsize = (12, 4))
chart = fig.add_subplot(1,1,1)

# 8개의 지점을 각각의 시계열 그래프로 플롯
for x in list_lc:
  chart.plot(df_202306_hw[df_202306_hw['지점'] == x]["일시"], df_202306_hw[df_202306_hw['지점'] == x]["최고기온(°C)"], label=x)
plt.legend(loc = 'best')

변수제외
df.columns.difference([v1, v2,...])

여러조건
df[(조건1) & (조건2)]